# Project 3: Analysing the stars

**It is an explicit requirement that you should <u>not</u> be using machine learning in the project element of this course.**

The Sloan Digital Sky Survey is a massive astronomy project that has been collecting images and optical spectra of objects in the night sky. It is used by researchers, but all the data is publicly available. One can deduce certain things from the spectra or associated information about the objects. For example, one can determine the redshift of a galaxy from its spectrum. This is a measure of how fast the galaxy is moving away from us. The spectra can also be used to determine the chemical composition of stars and galaxies.

<a href="https://www.sdss3.org/dr14/ ">Project page </a> 

In order to come up with project scenarios we highly recommend reading through a few of the basic, advanced and research challenge sections on this <a href="https://skyserver.sdss.org/dr10/en/proj/advanced/advancedhome.aspx ">page of tutorials</a>.

You don't necessarily need to solve these challenges, but they will give you some basic background and a sense of what kind of project might make sense. 

A challenge of working with this Astronomy dataset is the sheer size of it. There are hundreds of millions of objects in the database and so one cannot just blindly ask the server to send you images or spectra for all of them! Stars or Galaxies can also be given catchy names like NGC 5406 which are not things you are likely to know! Once you've come up with a scenario we suggest using the following <a href="https://skyserver.sdss.org/dr14/en/tools/search/form/searchform.aspx ">search tool</a> to narrow down objects that you want to work with. You should be aware that only a small fraction of the objects for which pictures exist have spectra (it is still a lot) but you can filter with this tool. You could download a manageable number of object ids and coordinates to a small csv, but be sure to include it with your project files so that we can reproduce your results.

The coordinate system for identifying where objects are in the sky frequently refers to ra and dec which can be a bit confusing. See <a href="https://solarsystem.nasa.gov/basics/chapter2-2/  ">this page</a> for a simple explanation. 

Example Scenarios:

1) An astronomer is interested in studying different types of astronomical objects. The optical spectrum of a quasar looks different to that of a galaxy or star. Young stars are very rich in elemental Hydrogen. Could you write a program that could classify objects based on their spectra? Perhaps given a list of objects it could return images of the objects suspected to be quasars or perhaps given a picture of a small bit of the night sky the astronomer could click on different objects and a label would appear telling them what type of object it is.

2) Build a star thermometer. Although there are thousands of spectra the SDSS survey has a much larger collection of images taken through a number of different filters. These filters are designed to only let certain wavelengths of light through. The u filter lets through ultraviolet light, the g filter lets through green light, the r filter lets through red light etc. The temperature of a star determines how much light it emits at different wavelengths. A hot star will emit more light in the ultraviolet than a cooler star. A cooler star will emit more light in the infrared than a hotter star. A  bit more background is explained in <a href="https://skyserver.sdss.org/dr10/en/proj/basic/color/colorandtemp.aspx">this tutorial </a>
Star spectrums are in many cases approximated quite well by a <a href="https://en.wikipedia.org/wiki/Black-body_radiation ">blackbody spectrum</a> with a known shape that depends on the Temperature. Can you write a tool that when given a star's object id will return the temperature of the star?

A Conda environment file project.yml is included to get you up and running.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Libraries to download data from the web
from PIL import Image
import requests
from urllib.parse import urlencode

#Some astropy submodules https://docs.astropy.org/en/stable/  
from astropy import units
from astropy import coordinates as coords

# astroquery module https://astroquery.readthedocs.io/en/latest/#introduction
# SDSS is a survey of the whole sky https://www.sdss.org/dr18/completed_surveys/
from astroquery.sdss import SDSS

In [ ]:
object_name = 'NGC 5406'
object = coords.SkyCoord.from_name(object_name)

print(object)
# Coordinate systems ra and dec - 
pos = coords.SkyCoord(object.ra, object.dec, frame='icrs')

# Sets size of image
im_size = 5*units.arcmin
im_pixels = 1024

# We will get an image from the Sloan Digital Sky Survey
# this builds the url to download the image
baseurl = 'http://skyservice.pha.jhu.edu/DR14/ImgCutout/getjpeg.aspx'
query_string = urlencode(dict(ra=object.ra.deg,
                              dec=object.dec.deg,
                              width=im_pixels, height=im_pixels,
                              scale=im_size.to(units.arcsec).value/im_pixels))
url = baseurl + '?' + query_string

# this downloads the image
response = requests.get(url, stream=True)
response.raw.decode_content = True  

# this displays the image
with Image.open(response.raw) as img:
    img = np.array(img)
    plt.imshow(img)

In [ ]:
# Passing the coordinates of the galaxy we look for a spectrum in the SDSS database
# https://astroquery.readthedocs.io/en/latest/sdss/sdss.html
search_radius = 0.1*units.arcmin
xid = SDSS.query_region(pos, radius=search_radius, spectro=True)
print(xid.colnames)
print('coords = ', xid['ra'][0], xid['dec'][0])
print('redshift = ', xid['z'][0])

spectra = SDSS.get_spectra(matches=xid)

#Find the magnitude of object taken through green filter
result = SDSS.query_crossid(pos, photoobj_fields=['modelMag_g'])
# print out the magnitudes taken through filters g and i
print('Magnitude green = ', result['modelMag_g'][0])

# Plotting the spectrum
spectra_data = spectra[0][1].data
plt.plot(10**spectra_data['loglam'], spectra_data['flux'])
plt.xlabel('wavelength (Angstrom)')
plt.ylabel('flux (nanomaggies)')
plt.title('SDSS spectra of '+ object_name)